## Parametros

In [5]:
codigo_canhero = 40132

## Librerias

In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

## Conectar a AmigoCloud

In [3]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Datos de crédito

In [6]:
query_credito = {'query': 'select * from dataset_282969 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_credito)
credito_data = credito_select['data'][0]
credito_data['fecha_informe'] = (datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d') + timedelta(days=1)).strftime('%d/%m/%Y')
credito_data['fecha_1ra_insp'] = datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [ ]:
credito

## Datos de propiedades

In [7]:
query_propiedades = {'query': 'select * from dataset_282968 where cre_ref_prop = \'{cre_ref_prop}\''.format(cre_ref_prop=credito.amigo_id)}
propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_propiedades)
propiedades_data = propiedades_select['data']
print('end')

end


In [8]:
propiedades_data

[{'ubicacion': '0101000020E6100000D6216A7BB9A74FC0D8EBE507E7EF30C0',
  'propiedad': '1755 / SANTA MARIA--VELARDE OSCAR',
  'fecha_registro': '2022-05-28 14:15:44+00:00',
  'fecha_actualizacion': '2022-05-28 14:15:44+00:00',
  'cre_ref_prop': '1863c049b4b94d80a7d0cb9763636c64',
  'amigo_id': 'b3a6e27c23d848bc9bba52c9730cc338',
  'tenencia': 'PROPIO'}]

## Datos de lotes

In [9]:
propiedades = []
for prop in propiedades_data:
    prop['lotes'] = []
    query_lotes = {'query': 'select * from dataset_282975 where prop_ref_lote=\'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
    lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_lotes)
    for lote_select in lotes_select['data']:
        lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
        prop['lotes'].append(lote)
    propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
print('end')

end


In [10]:
propiedades

[propiedad(ubicacion='0101000020E6100000D6216A7BB9A74FC0D8EBE507E7EF30C0', propiedad='1755 / SANTA MARIA--VELARDE OSCAR', fecha_registro='2022-05-28 14:15:44+00:00', fecha_actualizacion='2022-05-28 14:15:44+00:00', cre_ref_prop='1863c049b4b94d80a7d0cb9763636c64', amigo_id='b3a6e27c23d848bc9bba52c9730cc338', tenencia='PROPIO', lotes=[lote(prop_ref_lote='b3a6e27c23d848bc9bba52c9730cc338', geometria='0106000020E6100000010000000103000000010000000A000000CFA80DCC5BA74FC03F8074E9A6F030C0786E300D46A74FC0B8DF6E50BEF030C05BADFE3840A74FC0D4240FDBC9F030C0617D369945A74FC0DD7F1CA7F6F030C0212DA73736A74FC0B52B4E3F07F130C08ACD441E1FA74FC0107F0BCB1DF130C02234534E30A74FC0E9A2273268F130C0C4C5DE7272A74FC0683A24FD14F130C0D36388C365A74FC0C8C2BC1DD3F030C0CFA80DCC5BA74FC03F8074E9A6F030C0', fecha_registro='2022-08-05 17:21:42+00:00', lote='L2.1', superficie=4.15, textura='F', lote_semillero=None, variedad_semillero=None, variedad_sugerida=None, estado_semillero=None, mes_siembra=None, variedad_sembrada='RBB 77-

## Fotos

In [ ]:
# fotos genereles
query_fotos = {'query': 'select s3_filename from gallery_39467 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=credito.amigo_id)}
fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_fotos)
fotos_filename['data']
fotos = []
for file_name in fotos_filename['data']:
    fotos.append(collections.namedtuple("foto", file_name.keys())(*file_name.values()))
print('end')

In [ ]:
# fotos lotes
fotos = []
for propiedad in propiedades:
    for lote in propiedad.lotes:
        print(lote.lote, lote.amigo_id)
        query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_39467 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
        fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_fotos)
        if fotos_filename['data'] != []:
            dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
            fotos.append(dic)

fotos

## Recorrido de las propiedades y lotes

In [ ]:
for prop in propiedades:
    print(prop.propiedad)
    for lote in prop.lotes:
        print(lote.lote)

# Sección de planos

# ------------------------------------------------------------------------------------------

In [ ]:
from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

In [ ]:
i = 0
lista_planos = []
ruta = ''
for propiedad in propiedades:
    i = i + 1
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta = 'planos/' + str(credito.amigo_id) + '_' + str(propiedad.amigo_id) + '.jpeg'
    lista_planos.append(ruta)
    fig.savefig(ruta, dpi = 300, bbox_inches='tight')
    del fig
print('end')

In [ ]:
fotos

In [ ]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/template_word_1ra.docx")

#generar lista de InlineImage de planos 
lista_InlineImage = []
for plano in lista_planos:
    lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    file = open('fotos/' + foto['file_name'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})

firma_respon = None
if credito.respon_1ra_insp == 'Rogelio Acuña Rodríguez':
    firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_rogelio.png', width=Mm(60))
else:
    firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_jaldin.png', width=Mm(60))

context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}

doc.render(context)
file_name = credito.canhero.replace(' / ', '_')
doc.save('_' + file_name + '.docx')